In [1]:
import pandas as pd

In [2]:
importacoes_mundo_2023 = pd.read_excel('./dados/comex/Importação Mundo 2024 NCM.xlsx',dtype={'Código NCM':str})


In [8]:
importacoes_mundo_2023.head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,8690210062
1,27101921,Gasóleo (óleo diesel),8359492092
2,84119100,Partes de turborreatores ou de turbopropulsores,4837676499
3,31042090,Outros cloretos de potássio,3606983824
4,84111200,Turborreatores de empuxo superior a 25 kN,3418099488


In [5]:
tradutor_ncm = pd.read_excel('./dados/mip/TradutorNCM.xls', dtype={'Código NCM': str, 'Código Produto': str})

In [12]:
importacoes_mapeadas_scn = importacoes_mundo_2023.merge(tradutor_ncm.drop('Nome NCM',axis=1), how='left', left_on='Código NCM', right_on='Código NCM', suffixes=('', '_tradutor'), validate='1:1',indicator=True)

In [17]:
importacoes_nao_mapeadas = importacoes_mapeadas_scn.query('_merge == "left_only"')

In [20]:
lista_posicao_nao_mapeadas = importacoes_nao_mapeadas['Código NCM'].str[0:4].drop_duplicates().to_list()

In [23]:
importacoes_mapeadas_scn['Posicao NCM'] = importacoes_mapeadas_scn['Código NCM'].str[0:4]


In [26]:
importacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().sort_values(by='Posicao NCM')

,Posicao NCM,Código Produto,Descrição Produto
2449,0207,10913,Carne de aves
4461,2404,12001,Produtos do fumo
1275,3207,20923,"Tintas, vernizes, esmaltes e lacas"
194,3824,20922,Produtos químicos diversos
1620,3824,20921,Defensivos agrícolas e desinfestantes domissan...
2336,3824,20912,Adubos e fertilizantes
5744,3824,23002,"Artefatos de cimento, gesso e semelhantes"
1891,3827,20922,Produtos químicos diversos
411,3906,20914,"Resinas,elastômeros e fibras artif. e sintéticas"
802,4811,17002,"Papel, papelão, embalagens e artefatos de papel"


In [34]:
substituicao_valores_nulos = importacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` not in ["3824","7315"]')

In [29]:
importacoes_mapeadas_scn.query("_merge == 'left_only'").query('`Posicao NCM` == "3824"').head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Código Produto,Descrição Produto,_merge,Posicao NCM
3189,38249961,Contrastes para exames de diagnóstico por imag...,5624560,NaN,NaN,left_only,3824
7856,38249969,Outros contrastes para exames de diagnóstico p...,22508,NaN,NaN,left_only,3824


In [37]:
substituicao_valores_nulos = pd.concat([substituicao_valores_nulos, importacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` == "3824" and `Código Produto` == "20922"')])

In [30]:
importacoes_mapeadas_scn.query("_merge == 'left_only'").query('`Posicao NCM` == "7315"').head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Código Produto,Descrição Produto,_merge,Posicao NCM
2852,73151190,Outras correntes de rolos,7278774,NaN,NaN,left_only,7315
4236,73151110,Correntes de rolos do tipo utilizado em bicicl...,2429625,NaN,NaN,left_only,7315


In [39]:
substituicao_valores_nulos = pd.concat([substituicao_valores_nulos, importacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)')[['Posicao NCM','Código Produto', 'Descrição Produto']].drop_duplicates().query('`Posicao NCM` == "7315" and `Código Produto` == "25001"')])

In [40]:
#substituir apenas os valores nulos de acordo com a tabela de substituição e manter os demais valores sem alteração
importacoes_mapeadas_scn['Descrição Produto'] = importacoes_mapeadas_scn['Descrição Produto'].where(~importacoes_mapeadas_scn['Posicao NCM'].isin(lista_posicao_nao_mapeadas), importacoes_mapeadas_scn['Descrição Produto'].fillna(substituicao_valores_nulos.set_index('Posicao NCM')['Descrição Produto']))
importacoes_mapeadas_scn['Código Produto'] = importacoes_mapeadas_scn['Código Produto'].where(~importacoes_mapeadas_scn['Posicao NCM'].isin(lista_posicao_nao_mapeadas), importacoes_mapeadas_scn['Código Produto'].fillna(substituicao_valores_nulos.set_index('Posicao NCM')['Código Produto']))



In [42]:
#Verifica se a substituição foi feita corretamente
importacoes_mapeadas_scn.query('_merge == "both"').query('`Posicao NCM`.isin(@lista_posicao_nao_mapeadas)').drop_duplicates()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Código Produto,Descrição Produto,_merge,Posicao NCM
37,85044090,Outros conversores elétricos estáticos,884887236,27001,"Máquinas, aparelhos e materiais elétricos",both,8504
194,38249989,Outros produtos e preparações à base de compos...,211278154,20922,Produtos químicos diversos,both,3824
223,85044021,"Retificadores, exceto carregadores de acumulad...",187355703,27001,"Máquinas, aparelhos e materiais elétricos",both,8504
308,85044050,"Conversores eletrônicos de frequência, para va...",137718673,27001,"Máquinas, aparelhos e materiais elétricos",both,8504
338,55101111,"Fios que contenham pelo menos 85 %, em peso, d...",127053917,13001,Fios e fibras têxteis beneficiadas,both,5510
...,...,...,...,...,...,...,...
8365,38249933,Misturas e preparações para borracha ou plásti...,2246,20922,Produtos químicos diversos,both,3824
8445,38249943,"Misturas à base de trimetil-3,9-dietildecano",1266,20922,Produtos químicos diversos,both,3824
8465,38249911,Salinomicina micelial,1132,20922,Produtos químicos diversos,both,3824
8515,38249974,Preparações à base de hidróxido de níquel ou d...,788,20922,Produtos químicos diversos,both,3824


In [43]:
importacoes_mapeadas_scn.drop(columns=['_merge'], inplace=True)

In [44]:
#Verificar a participação de cada NCM em relação ao total de importações de um mesmo código de produto, considerando que todos os códigos de produto são válidos
importacoes_mapeadas_scn['Total Importações Produto'] = importacoes_mapeadas_scn.groupby('Código Produto')['2024 - Valor US$ FOB'].transform('sum')
importacoes_mapeadas_scn['Participação NCM no Produto'] = importacoes_mapeadas_scn['2024 - Valor US$ FOB'] / importacoes_mapeadas_scn['Total Importações Produto']



In [46]:
importacoes_mapeadas_scn.tail()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Código Produto,Descrição Produto,Posicao NCM,Total Importações Produto,Participação NCM no Produto
8743,94011010,"Assentos ejetáveis, dos tipos utilizados em ve...",5,30001,"Aeronaves, embarcações e outros equipamentos d...",9401,1.447465e+10,3.454314e-10
8744,03091000,"Farinhas, pós e ""pellets"" de peixes, para alim...",4,10914,Pescado industrializado,0309,7.377429e+08,5.421943e-09
8745,28121400,Pentacloreto de fósforo,3,20911,Produtos químicos inorgânicos,2812,1.435101e+10,2.090445e-10
8746,49070010,Papéis-moeda,2,18001,Serviços de impressão e reprodução,4907,5.012540e+07,3.989993e-08
8747,29242945,Iodamida,1,20913,Produtos químicos orgânicos,2924,1.236260e+10,8.088915e-11
